# Regional means

This notebook should demonstrate how CORDEX datasets can be used for looking at regional and local scales. This often involves masking and averaging over a limited area of the dataset, e.g., a country or city.

In [ ]:
import os
import xarray as xr
import numpy as np
import cf_xarray as cfxr
import pyesgf
from pyesgf.logon import LogonManager
from pyesgf.search import SearchConnection

lm = LogonManager(verify=True)

if not lm.is_logged_on():
    myproxy_host = "esgf-data.dkrz.de"
    # if we find those in environment, use them.
    if "ESGF_USER" in os.environ and "ESGF_PASSWORD" in os.environ:
        lm.logon(
            hostname=myproxy_host,
            username=os.environ["ESGF_USER"],
            password=os.environ["ESGF_PASSWORD"],
            interactive=False,
            bootstrap=True,
        )
    else:
        lm.logon(
            hostname=myproxy_host,
            interactive=True,
            bootstrap=True,
        )

print(f"logged on: {lm.is_logged_on()}")

In [ ]:
hist_urls = [
    "http://esgf1.dkrz.de/thredds/dodsC/cordex-reklies/output/EUR-11/CLMcom/MIROC-MIROC5/historical/r1i1p1/CLMcom-CCLM4-8-17/v1/mon/tas/v20171121/tas_EUR-11_MIROC-MIROC5_historical_r1i1p1_CLMcom-CCLM4-8-17_v1_mon_195001-195012.nc",
    "http://esgf1.dkrz.de/thredds/dodsC/cordex-reklies/output/EUR-11/CLMcom/MIROC-MIROC5/historical/r1i1p1/CLMcom-CCLM4-8-17/v1/mon/tas/v20171121/tas_EUR-11_MIROC-MIROC5_historical_r1i1p1_CLMcom-CCLM4-8-17_v1_mon_195101-196012.nc",
    "http://esgf1.dkrz.de/thredds/dodsC/cordex-reklies/output/EUR-11/CLMcom/MIROC-MIROC5/historical/r1i1p1/CLMcom-CCLM4-8-17/v1/mon/tas/v20171121/tas_EUR-11_MIROC-MIROC5_historical_r1i1p1_CLMcom-CCLM4-8-17_v1_mon_196101-197012.nc",
    "http://esgf1.dkrz.de/thredds/dodsC/cordex-reklies/output/EUR-11/CLMcom/MIROC-MIROC5/historical/r1i1p1/CLMcom-CCLM4-8-17/v1/mon/tas/v20171121/tas_EUR-11_MIROC-MIROC5_historical_r1i1p1_CLMcom-CCLM4-8-17_v1_mon_197101-198012.nc",
    "http://esgf1.dkrz.de/thredds/dodsC/cordex-reklies/output/EUR-11/CLMcom/MIROC-MIROC5/historical/r1i1p1/CLMcom-CCLM4-8-17/v1/mon/tas/v20171121/tas_EUR-11_MIROC-MIROC5_historical_r1i1p1_CLMcom-CCLM4-8-17_v1_mon_198101-199012.nc",
    "http://esgf1.dkrz.de/thredds/dodsC/cordex-reklies/output/EUR-11/CLMcom/MIROC-MIROC5/historical/r1i1p1/CLMcom-CCLM4-8-17/v1/mon/tas/v20171121/tas_EUR-11_MIROC-MIROC5_historical_r1i1p1_CLMcom-CCLM4-8-17_v1_mon_199101-200012.nc",
    "http://esgf1.dkrz.de/thredds/dodsC/cordex-reklies/output/EUR-11/CLMcom/MIROC-MIROC5/historical/r1i1p1/CLMcom-CCLM4-8-17/v1/mon/tas/v20171121/tas_EUR-11_MIROC-MIROC5_historical_r1i1p1_CLMcom-CCLM4-8-17_v1_mon_200101-200512.nc",
]


rcp85_urls = [
    "http://esgf1.dkrz.de/thredds/dodsC/cordex-reklies/output/EUR-11/CLMcom/MIROC-MIROC5/rcp85/r1i1p1/CLMcom-CCLM4-8-17/v1/mon/tas/v20171121/tas_EUR-11_MIROC-MIROC5_rcp85_r1i1p1_CLMcom-CCLM4-8-17_v1_mon_200601-201012.nc",
    "http://esgf1.dkrz.de/thredds/dodsC/cordex-reklies/output/EUR-11/CLMcom/MIROC-MIROC5/rcp85/r1i1p1/CLMcom-CCLM4-8-17/v1/mon/tas/v20171121/tas_EUR-11_MIROC-MIROC5_rcp85_r1i1p1_CLMcom-CCLM4-8-17_v1_mon_201101-202012.nc",
    "http://esgf1.dkrz.de/thredds/dodsC/cordex-reklies/output/EUR-11/CLMcom/MIROC-MIROC5/rcp85/r1i1p1/CLMcom-CCLM4-8-17/v1/mon/tas/v20171121/tas_EUR-11_MIROC-MIROC5_rcp85_r1i1p1_CLMcom-CCLM4-8-17_v1_mon_202101-203012.nc",
    "http://esgf1.dkrz.de/thredds/dodsC/cordex-reklies/output/EUR-11/CLMcom/MIROC-MIROC5/rcp85/r1i1p1/CLMcom-CCLM4-8-17/v1/mon/tas/v20171121/tas_EUR-11_MIROC-MIROC5_rcp85_r1i1p1_CLMcom-CCLM4-8-17_v1_mon_203101-204012.nc",
    "http://esgf1.dkrz.de/thredds/dodsC/cordex-reklies/output/EUR-11/CLMcom/MIROC-MIROC5/rcp85/r1i1p1/CLMcom-CCLM4-8-17/v1/mon/tas/v20171121/tas_EUR-11_MIROC-MIROC5_rcp85_r1i1p1_CLMcom-CCLM4-8-17_v1_mon_204101-205012.nc",
    "http://esgf1.dkrz.de/thredds/dodsC/cordex-reklies/output/EUR-11/CLMcom/MIROC-MIROC5/rcp85/r1i1p1/CLMcom-CCLM4-8-17/v1/mon/tas/v20171121/tas_EUR-11_MIROC-MIROC5_rcp85_r1i1p1_CLMcom-CCLM4-8-17_v1_mon_205101-206012.nc",
    "http://esgf1.dkrz.de/thredds/dodsC/cordex-reklies/output/EUR-11/CLMcom/MIROC-MIROC5/rcp85/r1i1p1/CLMcom-CCLM4-8-17/v1/mon/tas/v20171121/tas_EUR-11_MIROC-MIROC5_rcp85_r1i1p1_CLMcom-CCLM4-8-17_v1_mon_206101-207012.nc",
    "http://esgf1.dkrz.de/thredds/dodsC/cordex-reklies/output/EUR-11/CLMcom/MIROC-MIROC5/rcp85/r1i1p1/CLMcom-CCLM4-8-17/v1/mon/tas/v20171121/tas_EUR-11_MIROC-MIROC5_rcp85_r1i1p1_CLMcom-CCLM4-8-17_v1_mon_207101-208012.nc",
    "http://esgf1.dkrz.de/thredds/dodsC/cordex-reklies/output/EUR-11/CLMcom/MIROC-MIROC5/rcp85/r1i1p1/CLMcom-CCLM4-8-17/v1/mon/tas/v20171121/tas_EUR-11_MIROC-MIROC5_rcp85_r1i1p1_CLMcom-CCLM4-8-17_v1_mon_208101-209012.nc",
    "http://esgf1.dkrz.de/thredds/dodsC/cordex-reklies/output/EUR-11/CLMcom/MIROC-MIROC5/rcp85/r1i1p1/CLMcom-CCLM4-8-17/v1/mon/tas/v20171121/tas_EUR-11_MIROC-MIROC5_rcp85_r1i1p1_CLMcom-CCLM4-8-17_v1_mon_209101-210012.nc",
]

In [ ]:
%time ds = xr.open_mfdataset(
    hist_urls + rcp85_urls,
    chunks={},
    parallel=False,
    combine="by_coords",
    data_vars="minimal",
    coords="minimal",
    compat="override",
).load()

We are interested in looking at the change in the temperature with respect to the reference period from 1981 to 2010.

In [ ]:
ds_change = ds.drop("time_bnds").copy()
%time ds_change["tas"] = ds_change.tas - ds_change.tas.sel(time=slice("1981", "2010")).mean(
    "time"
)

In [ ]:
ds_change

Define a plotting function

In [ ]:
import cartopy.crs as ccrs


def plot(
    da,
    transform=ccrs.PlateCarree(),
    projection=ccrs.PlateCarree(),
    vmin=None,
    vmax=None,
    cmap=None,
    borders=True,
    xlocs=range(-180, 180, 2),
    ylocs=range(-90, 90, 2),
    extent=None,
    figsize=(15, 10),
    title=None,
):
    """plot a domain using the right projections and transformations with cartopy"""
    import cartopy.feature as cf
    import matplotlib.pyplot as plt

    plt.figure(figsize=figsize)
    ax = plt.axes(projection=projection)
    if extent:
        # ax.set_extent([ds_sub.rlon.min(), ds_sub.rlon.max(), ds_sub.rlat.min(), ds_sub.rlat.max()], crs=transform)
        ax.set_extent(extent, crs=projection)
    ax.gridlines(
        draw_labels=True, linewidth=0.5, color="gray", xlocs=xlocs, ylocs=ylocs
    )
    da.plot(ax=ax, cmap=cmap, transform=transform, vmin=vmin, vmax=vmax)
    ax.coastlines(resolution="50m", color="black", linewidth=1)
    if borders:
        ax.add_feature(cf.BORDERS)
    if title is not None:
        ax.set_title(title)

## Masking with `regionmask`

In [ ]:
import regionmask

In [ ]:
prudence = regionmask.defined_regions.prudence
prudence

Let's have a look at those regions:

In [ ]:
import matplotlib.pyplot as plt
from cartopy import crs as ccrs

plt.rcParams["figure.figsize"] = (20, 10)
pole = (
    ds.rotated_pole.grid_north_pole_longitude,
    ds.rotated_pole.grid_north_pole_latitude,
)
proj = ccrs.RotatedPole(*pole)
ax = prudence.plot(
    add_ocean=True,
    projection=proj,
    resolution="50m",
    label="name",
    line_kws=dict(lw=0.75),
)

Create a mask for each region that fits our dataset coordinates.

In [ ]:
mask_prudence = prudence.mask_3D(ds.lon, ds.lat)

We can now use the mask to only look at a certain region, e.g.

In [ ]:
me_tas = ds.tas.isel(time=0).where(
    mask_prudence.isel(region=(mask_prudence.names == "Mid-Europe")).squeeze(),
    drop=True,
)
plot(
    me_tas - 273.5,
    transform=ccrs.RotatedPole(*pole),
    projection=ccrs.RotatedPole(*pole),
)

Let's compute a spatial average over each region to compare the changes between them

In [ ]:
weights = np.cos(np.deg2rad(ds.rlat))
%time tas_change_regional = ds_change.tas.weighted(mask_prudence * weights).mean(
    dim=("rlat", "rlon")
)

The computation so far was done [lazily](https://docs.xarray.dev/en/stable/user-guide/dask.html). Before plotting, we will explictly trigger the compuation here.

In [ ]:
tas_change_regional.to_netcdf("tas_change_regional.nc")

Let's look at the change in yearly means for each region! We will use a rolling mean over the climatic timescale of 30 years.

In [ ]:
tas_change_regional.groupby("time.year").mean("time").rolling(year=30).mean().swap_dims(
    {"region": "names"}
).plot(hue="names")

In [ ]:
def get_grid(domain):
    """create cordex grid with bounds

    workaround for https://github.com/xarray-contrib/cf-xarray/issues/360

    """
    ds = cx.cordex_domain(domain, add_vertices=True)
    ds = ds.assign(
        lon_b=cfxr.bounds_to_vertices(ds.lon_vertices, "vertices"),
        lat_b=cfxr.bounds_to_vertices(ds.lat_vertices, "vertices"),
    ).drop(("lon_vertices", "lat_vertices"))
    ds.lon.attrs["bounds"] = "lon_b"
    ds.lat.attrs["bounds"] = "lat_b"
    return ds


def get_averager(grid, gdf):
    """create spatial averager for a grid file"""
    return xe.SpatialAverager(grid, gdf.geometry, geom_dim_name="region")